In [5]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."

import json
import os
from ibm_watson import CompareComplyV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException

import pandas as pd 
import text_extensions_for_pandas as tp
from IPython.core.display import HTML



## Initialize data and pre-process through watson Compare & Comply API

In [2]:
def init_watson_table_api():
    # Retrieve the APIKEY for authentication
    apikey = os.environ.get("IBM_API_KEY_TABLES")
    if apikey is None:
        raise ValueError("Expected apikey in the environment variable 'IBM_API_KEY'")

    # Get the service URL for your IBM Cloud instance
    ibm_cloud_service_url = os.environ.get("IBM_SERVICE_URL_TABLES")
    if ibm_cloud_service_url is None:
        raise ValueError("Expected IBM cloud service URL in the environment variable 'IBM_SERVICE_URL'")

        #initialize the authenticator
    authenticator = IAMAuthenticator(apikey)
    compare_comply = CompareComplyV1(
        version = '2020-1-1',
        authenticator=authenticator
    )

    compare_comply.set_service_url(ibm_cloud_service_url)
    return compare_comply

In [7]:
base_example_path = "../resources/tables/"
file_names = ["20-populous-countries", "cali-temp-chart", "california-population-chart", "double_header_table", "who_covid_report_table"]

archive_name = "archive"
#set to true to query watson
re_query_watson = False



In [14]:
#load in files either by querying cloud or from archive. 
if re_query_watson:
    print("This shouldn't happen")
    compare_comply = init_watson_table_api()
    for file_n in file_names:
        with open(f"{base_example_path}{file_n}.pdf", 'rb') as base_file:
            try:
                result = compare_comply.extract_tables(base_file).get_result()
                responses_dict[file_n] = result
            except ApiException as ex:
                print ("Method failed with status code " + str(ex.code) + ": " + ex.message)
     #archive file
    with open(f"{base_example_path}{archive_name}.json", 'w') as archive_file:
        json.dump(responses_dict, archive_file)
else:
    with open(f"{base_example_path}{archive_name}.json", 'r') as archive_file:
        responses_dict = json.load(archive_file)

## Now demo table extraction on a number of different tables.
To show how this tool can work in a variety of circumstances, here we demo using it on a number of different tables. 

First we display the reconstructed table, then we display the html that is created as an intermediate step of the table extraction, for comparison 

In [19]:
for name in file_names:
    print(f"\n\n Displaying table {name}:")
    print("\n Displaying Dataframe: \n")
    parsed = tp.watson_tables_parse_response(responses_dict[name])
    display(tp.make_table(parsed))
    print("\n\n\n HTML: \n\n")
    display(HTML(responses_dict[name]["document"]["html"]))




 Displaying table 20-populous-countries:

 Displaying Dataframe: 



,% of worldpopulation,Country (or\ndependent\nterritory),Date,Population,Rank,Source
1,18.0%,China [b],21 Jul 2020,"1,403,627,360",1,National populationclock [3]
2,17.5%,India [c],21 Jul 2020,"1,364,965,498",2,National populationclock [4]
3,4.23%,United States [d],21 Jul 2020,"329,991,308",3,National population\nclock [5]
4,3.46%,Indonesia,1 Jul 2020,"269,603,400",4,National annualprojection [6]
5,2.83%,Pakistan [e],1 Jul 2020,"220,892,331",5,UN Projection [2]
6,2.72%,Brazil,21 Jul 2020,"211,822,143",6,National populationclock [7]
7,2.64%,Nigeria,1 Jul 2020,"206,139,587",7,UN Projection [2]
8,2.17%,Bangladesh,21 Jul 2020,"168,990,780",8,National populationclock [8]
9,1.88%,Russia [f],1 Jan 2020,"146,748,590",9,National estimate [9]
10,1.64%,Mexico,1 Jul 2020,"127,792,286",10,National annualprojection [10]





 HTML: 




Rank,Country (or dependent territory),Population,% of worldpopulation,Date,Source
1,China[b],"1,403,627,360",18.0%,21 Jul 2020,National populationclock[3]
2,India[c],"1,364,965,498",17.5%,21 Jul 2020,National populationclock[4]
3,United States[d],"329,991,308",4.23%,21 Jul 2020,National population clock[5]
4,Indonesia,"269,603,400",3.46%,1 Jul 2020,National annualprojection[6]
5,Pakistan[e],"220,892,331",2.83%,1 Jul 2020,UN Projection[2]
6,Brazil,"211,822,143",2.72%,21 Jul 2020,National populationclock[7]
7,Nigeria,"206,139,587",2.64%,1 Jul 2020,UN Projection[2]
8,Bangladesh,"168,990,780",2.17%,21 Jul 2020,National populationclock[8]
9,Russia[f],"146,748,590",1.88%,1 Jan 2020,National estimate[9]
10,Mexico,"127,792,286",1.64%,1 Jul 2020,National annualprojection[10]




 Displaying table cali-temp-chart:

 Displaying Dataframe: 



...

Nine months ended setptember 30         \
                                                2004   2005   
Dividends received                              4.7%  15.4%   
IRS audit settlement                           15.2%    58%   
Statatory tax rate                               38%    37%   
Total tax rate                                 15.1%  38.8%   

                     Three months ended setptember 30         
                                                 2004   2005  
Dividends received                               3.3%  13.2%  
IRS audit settlement                            35.5%    97%  
Statatory tax rate                                36%    35%  
Total tax rate                                   4.3%  76.1%




 HTML: 






 Displaying table who_covid_report_table:

 Displaying Dataframe: 



Days since last Reporting Country/ Territory/Area Total confirmed  \
     reported case                                             cases   
2                                             Africa                   
3                0                      South Africa         287 796   
4                0                           Nigeria        33 | 153   
5                0                             Ghana        24 | 988   
6                0                           Algeria        19 | 689   
7                1                          Cameroon        15 | 173   
8                0                     Côƚe d͛Iǀoiƌe        12 | 872   
9                0                             Kenya        10 | 294   
10               0                           Senegal         8 | 198   
11               0  Democratic Republic of the Congo         8 | 074   
12               0                          Ethiopia         7 | 969   
13               0                            Guinea         6 | 170   
14               3                             Gabon         5 | 942   
15               0                        Mauritania         5 | 355   
16               0                        Madagascar         5 | 080   
17               0          Central African Republic         4 | 321   
18               0                            Malawi         2 | 430   
19               0                              Mali         2 | 412   
20               0                            Zambia         2 | 283   
21               0                             Congo         2 | 222   
22               1                       South Sudan         2 | 148   
23               0                     Guinea-Bissau         1 | 902   

             Total deaths Total new deaths            Transmission  
   new cases                                      classification i  
2                                                                   
3     11 554        4 172               93  Community transmission  
4        595          744                4  Community transmission  
5        470          139                0  Community transmission  
6        494        1 018                7  Community transmission  
7          0          359                0  Community transmission  
8        106           84                0  Community transmission  
9        189          197               12  Community transmission  
10        63          150                2  Community transmission  
11        42          189                1  Community transmission  
12       203          139               11  Community transmission  
13        29           38                1  Community transmission  
14         0           46                0  Community transmission  
15        80          147                0       Clusters of cases  
16       213           37                2       Clusters of cases  
17        33           53                0  Community transmission  
18        66           39                1       Clusters of cases  
19         1          121                0  Community transmission  
20       388           82               40  Community transmission  
21       119           47                0  Community transmission  
22         0           41                0       Clusters of cases  
23        60           26                1  Community transmission




 HTML: 


